In [37]:
# load all relevant packages and code

load('Falcon_stuff.sage')
load('New_Falcon_stuff.sage')
load('CBD_stuff.sage')

import gc

from Entropy_stuff import *
from Multinomial import *
from Largelog import *
from Root_sum import approx_sum_of_roots
from Compact_Dictionary import *

import gc

from Entropy_stuff import *
from Multinomial import *
from Largelog import *
from Root_sum import approx_sum_of_roots
from Compact_Dictionary import *

class distribution:
    def __init__(self, dist, base=2):
        cumu=0
        for key in dist:
            cumu+=dist[key]
        if cumu==0:
            print("Not a valid distribution")
        elif cumu!=1:
            self.d = {}
            for key in dist:
                self.d[key]=dist[key]/cumu
            self.dist=self.d
        else:
            self.d = dist
            self.dist = dist
        self.p, self.m=multiplicity(self.dist)
        self.label={}
        for prob in self.p:
            self.label[prob]=[]
            for key in self.dist:
                if self.dist[key]==prob:
                    self.label[prob]+=[key]
        eta=len(self.p)-1
        self.eta = eta
        self.range = list(self.d)
        self.log_p = []
        for prob in self.p:
            self.log_p += [-largelog(prob,2)]
        self.entropy = dist_entropy(self.dist, 1, base=base)
        self.comp_dics={}
        
    def __repr__(self, indent=10):
        l=str(indent)
        out="Distribution with sampling probabilities:\n"
        for key in self.label:
            b=str(self.label[key][0])
            for i in range(1,len(self.label[key])):
                b+=", " + str(self.label[key][i])
            out+='{0: >10}'.format(b) + " : " + str(key) + "\n"
        out+="\n"
        out+="Entropy of distribution is " + str(self.entropy)
        return out
    
    
    ### PROBABILITY FUNCTIONS ###
    
    def prob(self, i, f=False):
        # returns the probability of sampling i.
        if f:
            return float(self.d.get(i,0))
        return self.d.get(i,0)
    
    def vec_prob(self, v, f=False):
        # returns the probability of sampling a specific vector v=[i_1,...,i_n] from A^n.
        p=1
        for i in v:
            p*=self.prob(i, f=f)
        return p
        
    def compact_vec_prob(self, l, f=False):
        # returns the probability of sampling a specific vector with unsigned weight l=[n_0,...,n_eta]
        # (e.g. self.vec_prob([0,1,-1]) = self.compact_vec_prob([1,2]) ).
        p=0
        if len(l)-1<=self.eta:
            p=1
            for i in range(len(l)):
                p*=self.prob(i, f=f)**l[i]
        return p
    
    
    ### COMPACT DICTIONARY FUNCTIONS ###
    
    def comp_dic_list(self, n):
        # returns the compact dictionary for vectors of length n if it already exists; 
        # otherwise, returns an empty compact dictionary.
        return self.comp_dics.get(n,empty_comp_dic)
    
    def create_par_comp_dic(self, n, p_limit,cur_j,high_n,L,p, if_print=True):
        # iterative function for par_comp_dic and comp_dic.
        
        # iterates over all viable vector weights with sufficient sampling probability, gathers them in a list with
        # their sampling probability and their amount of occurrences(i.e. the amount of unsigned permutations)
        # and sorts that list by probability.
        if cur_j==self.eta and self.log_p[self.eta]*high_n<=p_limit:
            L[cur_j]=high_n
            if if_print:
                print(L)
            count=count_with_multiplicities(L, self.m)
            self.comp_dics[n].dic+=[[L.copy(),p*self.p[cur_j]**high_n,count]]
            self.comp_dics[n].count+=count
            self.comp_dics[n].p+=count*p*self.p[cur_j]**high_n
            L[cur_j]='-'
        elif high_n==0:
            for j in range(cur_j, self.eta+1):
                L[j]=0
            if if_print:
                print(L)
            count=count_with_multiplicities(L, self.m)
            self.comp_dics[n].dic+=[[L.copy(),p*self.p[cur_j]**high_n,count]]
            self.comp_dics[n].count+=count
            self.comp_dics[n].p+=count*p*self.p[cur_j]**high_n
            for j in range(cur_j, self.eta+1):
                L[j]='-'
        elif cur_j<self.eta:
            if p_limit//self.log_p[cur_j]>=high_n:
                for i in range(high_n+1):
                    L[cur_j]=i
                    self.create_par_comp_dic(n, p_limit-i*self.log_p[cur_j],cur_j+1,high_n-i,L,p*self.p[cur_j]**i, if_print=if_print)
    
    def par_comp_dic(self, n, c=0, if_ret=True, if_print=False):
        # creates a partial compact dictionary that contains all entries of the full compact dictionary which have
        # sampling probability of at least 2^(entropy*n+c).
        # If a partial compact dictionary already exists for a smaller c, the preexisting compact dictionary is used as
        # a base; 
        # otherwise, the old compact dictionary is overwritten
        if self.comp_dic_list(n).c<c:
            if self.eta==0:
                self.comp_dics[n]=par_comp_dic(c,[[n]])
            else:
                gc.collect()
                self.comp_dics[n]=par_comp_dic(-float('inf'),[],0,0)
                self.create_par_comp_dic(n,self.entropy*n+c,0,n,(self.eta+1)*['-'],1, if_print=if_print)
                self.comp_dics[n].dic.sort(key=lambda x: x[1], reverse=True)
                self.comp_dics[n].len=len(self.comp_dics[n].dic)
                self.comp_dics[n].c=c
                gc.collect()
                out=self.comp_dic_list(n)
        elif self.comp_dic_list(n).c>c:
            out_L=[]
            p=0
            count=0
            for l in self.comp_dic_list(n).dic:
                if largelog(l[1])<-self.entropy*n-c:
                    break
                out_L+=[l]
                p+=l[1]*l[2]
                count+=l[2]
            out=par_comp_dic(c, out_L, count, p)
        else:
            out=self.comp_dic_list(n)
        if if_ret:
            return out
        
    def comp_dic(self, n, if_ret=True, if_print=False):
        # creates the full compact dictionary
        c=-n*(self.entropy-max(self.log_p))+1
        return self.par_comp_dic(n, c=c, if_ret=if_ret, if_print=if_print)
    
    
    ### TABLE FUNCTIONS ###
    
    def raw_data(self, low_n, high_n, c=0, delete_after=False, step=1, aborts=True, if_parallel=False):
        # raw_data returns the success probability and complexities for all n in the range of [low_n, high_n]
        # in a csv-style table.
        
        # If aborts = True, the table is composed based on data from partial compact distionaries. This is faster,
        # but does not contain the expected runtime for the KeyGuess-Algorithm
        
        # If aborts = False, both expected data for KeyGuess and AbortedKeyGuess will be displayed, at the cost of
        # a significant runtime increase. Not recommended for wide distributions (like Falcon)
        if aborts:
            self.make_raw_data(low_n, high_n, c=c, delete_after=delete_after, step=step)
        else:
            self.make_raw_data_no_aborts(low_n, high_n, c=c, delete_after=delete_after, step=step, if_parallel=if_parallel)
            
    
    @parallel(256)
    def single_raw_data(self, n, c=0, delete_after=False, step=1, aborts=True):
        
        # raw_data returns the success probability and complexities for all n in the range of [low_n, high_n]
        # in a csv-style table.
        
        # If aborts = True, the table is composed based on data from partial compact distionaries. This is faster,
        # but does not contain the expected runtime for the KeyGuess-Algorithm
        
        # If aborts = False, both expected data for KeyGuess and AbortedKeyGuess will be displayed, at the cost of
        # a significant runtime increase. Not recommended for wide distributions (like Falcon)
        if aborts:
            L=self.par_comp_dic(n, c=c, if_ret=True, if_print=False)
            p=0
            count=0
            ET=0
            Equantum=0
            for l in L.dic:
                p+=l[1]*l[2]
                ET+=l[1]*l[2]*(count+(l[2]+1)/2)
                Equantum+=l[1]*approx_sum_of_roots(count+1,count+l[2])
                count+=l[2]
            ET+=(1-p)*count
            return(n,format(float(p),'f'),largelog(count),largelog(ET),largelog(Equantum))
            if delete_after:
                del(self.comp_dics[n])
                gc.collect()
        else:
            L=self.comp_dic(n, if_ret=True, if_print=False)
            p1=0
            p2=0
            count1=0
            count2=0
            ET1=0
            ET2=0
            Equantum=0
            for l in L.dic:
                if largelog(l[1])>=-self.entropy*n-c:
                    p1+=l[1]*l[2]
                    ET1+=l[1]*l[2]*(count1+(l[2]+1)/2)
                    Equantum+=l[1]*approx_sum_of_roots(count1+1,count1+l[2])
                    count1+=l[2]
#                else:
#                    ELZ1+=(1-p1)*l[2]
                p2+=l[1]*l[2]
                ET2+=l[1]*l[2]*(count2+(l[2]+1)/2)
                count2+=l[2]
            ET1+=(1-p1)*count1
            print(n,format(float(p1),'f'),largelog(count1),largelog(ET1),largelog(ET2),largelog(Equantum))
            if delete_after:
                del(self.comp_dics[n])
                gc.collect()
    
    def make_raw_data(self, low_n, high_n, c=0, delete_after=False, step=1, if_parallel=False):
        if if_parallel==False:
            print("n epsilon coresize Eclassic Equantum")
        for n in range(low_n, high_n+1, step):
            L=self.par_comp_dic(n, c=c, if_ret=True, if_print=False)
            p=0
            count=0
            ET=0
            Equantum=0
            for l in L.dic:
                p+=l[1]*l[2]
                ET+=l[1]*l[2]*(count+(l[2]+1)/2)
                Equantum+=l[1]*approx_sum_of_roots(count+1,count+l[2])
                count+=l[2]
            ET+=(1-p)*count
            print(n,format(float(p),'f'),largelog(count),largelog(ET),largelog(Equantum))
            if delete_after:
                del(self.comp_dics[n])
                gc.collect()
    
    def make_raw_data_no_aborts(self, low_n, high_n, c=0, delete_after=False, step=1):
        print("n epsilon coresize Eclassic Enoabort Equantum")
        for n in range(low_n, high_n+1, step):
            L=self.comp_dic(n, if_ret=True, if_print=False)
            p1=0
            p2=0
            count1=0
            count2=0
            ET1=0
            ET2=0
            Equantum=0
            for l in L.dic:
                if largelog(l[1])>=-self.entropy*n-c:
                    p1+=l[1]*l[2]
                    ET1+=l[1]*l[2]*(count1+(l[2]+1)/2)
                    Equantum+=l[1]*approx_sum_of_roots(count1+1,count1+l[2])
                    count1+=l[2]
#                else:
#                    ELZ1+=(1-p1)*l[2]
                p2+=l[1]*l[2]
                ET2+=l[1]*l[2]*(count2+(l[2]+1)/2)
                count2+=l[2]
            ET1+=(1-p1)*count1
            print(n,format(float(p1),'f'),largelog(count1),largelog(ET1),largelog(ET2),largelog(Equantum))
            if delete_after:
                del(self.comp_dics[n])
                gc.collect()
    
    
    ### OTHER FUNCTIONS ###
        
    def count_til_p(self,n,c=0,ret_l=True, if_print=True):
        # Returns the success probability of AbortedKeyGuess that occurs when AbortedKeyGuess is run
        # until the sampling probability goes below 2^(-H(.)n+c).
        if self.comp_dic_list(n).c<c:
            self.par_comp_dic(n,c,if_print=if_print)
            out=self.comp_dic_list(n).count
        elif self.comp_dic_list(n).c==c:
            out=self.comp_dic_list(n).count
        else:
            out=0
            i=0
            L=self.comp_dic_list(n).dic
            while i<len(L) and largelog(L[i][1])>-self.entropy*n-c:
                out+=L[i][2]
                i+=1
        if ret_l:
            return largelog(out)
        return out

In [38]:
def create_falcon_dist(sigma, denominator_goodness_ratio=2**(-7), nosamples=2**20, goodness_limit=0.999):
    # step 1: find an upper bound b_0 for the denominator of p_i (i.e. the sum of all exp(-j^2/(2*sigma^2)) ) that is in very close proximity
    
    b_0=0
    denominator_old=1
    denominator_new=1
    while denominator_new/denominator_old>denominator_goodness_ratio:
        denominator_old=denominator_new
        b_0+=1
        denominator_new=exp(-(b_0)^2/(2*sigma^2))
    denominator=0
    for i in range(-b_0,b_0+1):
        denominator+=exp(-i^2/(2*sigma^2))
    
    # step 2: find an upper bound b_1 such that sampling nosamples many coefficients according to our approximate FALCON distribution would return only return elements that are absolute smaller than b_1
    b_1=0
    p=1/denominator
    while p^nosamples<goodness_limit:
        b_1+=1
        p+=2*exp(-(b_1)^2/(2*sigma^2))/denominator
    
    # step 3: calculate actual denominator for truncated distribution
    true_denominator=1
    for i in range(1,b_1+1):
        true_denominator+=2*exp(-i^2/(2*sigma^2))
    
    # step 4: create distribution
    dist={}
    for i in range(-b_1,b_1+1):
        dist[i]=exp(-i^2/(2*sigma^2))/true_denominator
    return distribution(dist)
    



In [47]:
f1024=create_falcon_dist(falcon1024_sigma,denominator_goodness_ratio=2**(-10))
f512=create_falcon_dist(falcon512_sigma,denominator_goodness_ratio=2**(-10))

In [46]:
print("n epsilon coresize Eclassic Equantum")
for _,data in f1024.single_raw_data(list(range(1,51))):
    print(data[0], data[1], data[2], data[3], data[4])

n epsilon coresize Eclassic Equantum
1 0.619382 2.321928094887362 1.8793961398706078 0.019355428059633414
4 0.580875 12.3063465536885 11.642630120100163 4.502451560479973
5 0.585404 15.802844488420687 15.098744521417922 6.19381741019671
6 0.578380 19.247883485213144 18.53129699499643 7.846926476333045
7 0.571430 22.700856014841396 21.976033588535728 9.509728879899471
8 0.564781 26.160108715845645 25.429941132568572 11.180930963070436
2 0.612300 5.614709844115208 4.998576128536096 1.396200492323218
3 0.592002 8.92184093707449 8.277972139547694 2.9060026429588395
9 0.558748 29.626615833872236 28.892539649361595 12.860683312645584


In [48]:
print("n epsilon coresize Eclassic Equantum")
for _,data in f512.single_raw_data(list(range(1,11))):
    print(data[0], data[1], data[2], data[3], data[4])

n epsilon coresize Eclassic Equantum
1 0.734323 3.1699250014423126 2.5241578287082373 0.586728930826761
4 0.588176 14.348244039977779 13.670030611570684 5.534771681044587
5 0.587408 18.315481285939416 17.607185057543894 7.453055963096946
6 0.575557 22.23049927049313 21.519144184540014 9.332775937955857
7 0.567300 26.17275056379007 25.456013973320875 11.238193062872469
8 0.566575 30.17371523703035 29.439692240553153 13.190447824747187
9 0.560391 34.13961272031239 33.4020147519997 15.119735190885992
2 0.624528 6.6582114827517955 6.016977726204942 1.9296700725406277
3 0.601874 10.470658874060552 9.807839847796597 3.695159085078403
10 0.560539 38.159161884818175 37.40671342521579 17.089470927859022


In [147]:
distribution(falcon1024dist)

Distribution with sampling probabilities:
         0 : 72973/524288
     -1, 1 : 2151/16384
     -2, 2 : 113981/1048576
     -3, 3 : 84393/1048576
     -4, 4 : 55367/1048576
     -5, 5 : 3983/131072
     -6, 6 : 8099/524288
     -7, 7 : 3711/524288
     -8, 8 : 1465/524288
     -9, 9 : 1055/1048576
   -10, 10 : 307/1048576
   -11, 11 : 109/1048576
   -12, 12 : 11/524288
   -13, 13 : 3/1048576

Entropy of distribution is 3.565524024663833

In [112]:
bounded_gaussian(1,falcon1024_sigma).dist

{-1: 0.32650268007320027, 0: 0.34699463985359946, 1: 0.32650268007320027}

In [70]:
d_A=distribution(falcon512dist)

In [106]:
B2.raw_data(1,3)

n epsilon coresize Eclassic Equantum
1 0.875000 1.5849625007211563 1.0 0.21607124505355713
2 0.765625 3.1699250014423126 2.4429434958487284 0.599744272000731
3 0.669922 4.754887502163469 4.057314877782703 1.0849045981735483


In [14]:
float(PDF_gaussian(-1,200,4)+PDF_gaussian(0,200,4)+PDF_gaussian(1,200,4))

0.2930696285017828

In [17]:
ds={}
ds.get(1,1)
ds

{}

In [62]:
# Probability distributions are handled using dictionaries p, where the probability of sampling i is defined via
# p[i] (if i can be sampled)

B1_pdist={
    -1:1/4,
     0:2/4,
     1:1/4
}

In [ ]:
# The distribution class takes as input a probability distribution and optional value base, which denotes the
# entropy base. If unchanged, the latter is set to 2.

B1e=distribution(B1_pdist, base=e)
print(B1e)
print()

B1=distribution(B1_pdist)
print(B1)

In [ ]:
# If the input probability distribution is not normalized, distribution class automatically normalizes it

B2_pdist={
    -2:1,
    -1:4,
     0:6,
     1:4,
     2:1,
}

B2=distribution(B2_pdist)
B2

In [ ]:
# This program comes with 5 predefined distributions: B1, B2, B3, Falcon512 (~D(4.06)) and Falcon1024 (~D(2.87)).
# Additionally, other centered binomial distributions with parameter eta can be created via CBD(eta)

B10=CBD(10)
B10

In [ ]:
# The normalized probability distribution of a distribution class object can be called with self.dist or self.d .
# The entropy is returned with self.entropy .

print(B2.dist)
print(B2.entropy)

In [ ]:
# To see every unique probability value that exists in the distribution, call self.p (or self.log_p for
# their absolute log-values).

print(B2.p)
print(B2.log_p)

# To see how often these occur, call self.m; the latter is ordered such that self.p[i] appears self.m[i] many times:

print(B2.m)

# To see all possible sampling values, sorted by their probability of sampling, call self.label

print(B2.label)

In [ ]:
# The size of range of possible probabilities is denoted with eta. Since we generally deal
# with distributions that are centered around 0, this usually coincides with the sampling range [-eta , ... , eta].
# The range can be called with self.range

print(B2.eta)
print(B2.range)

In [ ]:
# To find the probability of sampling a certain i, call self.prob(i). Can also be called for elements not in
# the sampling range

print(B2.prob(2))
print(B2.prob('Hello World'))

# To find the probability of sampling a certain vector v, call self.vec_prob(v)

print(B2.vec_prob([-2, 1, 0, -1]))

# A more compact way of representing a vector (and its unsigned permutations) is by counting
# how often a certain position/ probability occurs. For example, [-2, 1, 0, -1] can be represented
# by counting every 0, every +1, -1 and every +2, -2 and putting these weights in the list l = [1, 2, 1]
# to find the probability of a vector with only stating its weights can be done with self.compact_vec_prob(l)

print(B2.compact_vec_prob([1, 2, 1]))

# each probability function has optional input f, which, if set to true, converts the output to float:

print(B2.compact_vec_prob([1, 2, 1], f = True))

In [ ]:
# The calculation of (expected) runtimes requires the ability to build compact dictionaries.
# To create a compact dictionary for dimension n, call self.comp_dic(n):

print(B2.comp_dic(4))

# Note that these dictionaries are of size O(n^eta), which can be very large for wide distributions
# (like D(4.06) or D(2.87)). To combat this, we can create partial compact dictionaries that only contain
# vectors above a certain probability threshold, say 2^(-H(.)n-c) for entropy H(.) and some constant c.
# To create such an partial compact dictionary, call self.par_comp_dic(n, c):

print(B2.par_comp_dic(5,2))
print(B2.par_comp_dic(5,0))
print(B2.par_comp_dic(5,-2))

# If a partial compact dictionary has been calculated previously for parameters (n, c) and the function is
# called again for (n, c') where c' < c, the former list can be reused to calculate the compact dictionary faster. 
# If c' > c, we have to restart the whole computation process. We can not use the previous partial compact dictionary.

In [ ]:
# distribution class objects contain a pointer to all their peviously created compact dictionaries. These
# pointers can be found in the dictionary self.comp_dics:

print(B2.comp_dics)

# To call a specific (partial) compact dictionary for dimension n, call self.comp_dic_list(n):

print(B2.comp_dic_list(5))

# If said compact dictionary has not yet been computed, this returns an empty compact dictionary instead:

print(B2.comp_dic_list(2**16))

In [ ]:
# Compact dictionaries are their own class. To access the actual dictionary, call self_cd.dic

ex_cd=B2.comp_dic_list(5)
ex_cd.dic

# every item in self_cd.dic is a list that contains 3 items: the actual weight distribution, the sampling distribution
# for a vector with said distribution and the amount of vectors that have this unsigned weight distribution.

In [ ]:
# To retreive the amount of vectors that are represented with the stored partial dictionary, call self_cd.count

print(ex_cd.count)

# The cumulative sampling probability of all these vectors can be returned with self_cd.p

print(ex_cd.p)

In [ ]:
# The value for c from function call self.par_comp_dic(n, c) is stored in self_cd.c

print(ex_cd.c)

# The empty compact dictionary has value c set to -inf. Compact dictionaries created with calling self.comp_dic(n)
# have their value of c set to -(H(.)-max(self.log_p))n+1

print(empty_comp_dic.c)
print(B2.comp_dic_list(4).c)

In [ ]:
# To create csv-style tables containing raw data for n in range [low_n, high_n], call self.raw_data():

B2.raw_data(1,3)
print()

# the column heads of the csv table are
# n: vector dimension n
# p: probability that a randomly sampled vector has sampling probability 2^(-H(.)n-c)
# coresize: amount of vectors that satisfy the above condition (i.e. size of core set)
# Eclassic: expected runtime of running AbortedKeyGuess on that set of vectors
# Equantum: expected runtime of running Montanaro's algorithm on the very set of vectors

# By default, we set c to 0. If another calue for c is required, c can optionally be altered:

B2.raw_data(1,3, c=3)
print()

# If not every element from that range is required, the step size can be increased with the optional step command:

B2.raw_data(1,5, step=2)
print()

# Since we do not need the complete compact dictionary except when we compare the expected runtimes of KeyGuess
# and AbortedKeyGuess, we omit the expected runtime of KeyGuess unless specifically asked for. This can be done
# with the optional command aborts = False:

B2.raw_data(1,3, aborts = False)
print()

# The last function call has an additional column that contains the expected runtime of KeyGuess with column head
# Enoabort: Expected runtime of KeyGuess

# If the compact dictionaries are no longer required after the csv table is computed, the optional command
# delete_after can be set to true to immediately delete the compact dictionaries:

print(list(B2.comp_dics))
B2.raw_data(1,3, delete_after = True)
print(list(B2.comp_dics))

# Note how the last call of B2.comp_dics does not contain the keys n = 1, 2, 3.